In [25]:
import pandas as pd
from textblob import TextBlob

In [8]:
#load data
data = pd.read_csv('../../Data/raw_analyst_ratings.csv')

In [12]:
data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


### **Descriptive Statistics**


In [14]:
headline_lengths = data['headline'].astype(str).str.len()  # Ensure headlines are strings
print("Headline Length Statistics:")
print(headline_lengths.describe())

Headline Length Statistics:
count    1.407328e+06
mean     7.312051e+01
std      4.073531e+01
min      3.000000e+00
25%      4.700000e+01
50%      6.400000e+01
75%      8.700000e+01
max      5.120000e+02
Name: headline, dtype: float64


In [15]:
publisher_counts = data['publisher'].value_counts()

print("\nNumber of articles per publisher:")
publisher_counts


Number of articles per publisher:


publisher
Paul Quintaro        228373
Lisa Levin           186979
Benzinga Newsdesk    150484
Charles Gross         96732
Monica Gerson         82380
                      ...  
MoneyGeek                 1
muathe                    1
Robert Morris             1
LeftCoastHedgie           1
Jeremie Capron            1
Name: count, Length: 1034, dtype: int64

In [22]:
data['date'] = pd.to_datetime(data['date'],format='%Y-%m-%d %H:%M:%S', errors='coerce')

daily_counts = data.groupby(data['date'].dt.date).size()
print("\nDaily News Frequency:\n", daily_counts)

dayofweek_counts = data.groupby(data['date'].dt.dayofweek).size()
print("\nDay-of-week Trends (0=Monday, 6=Sunday):\n", dayofweek_counts)

monthly_counts = data.groupby(data['date'].dt.to_period('M')).size()
print("\nMonthly Trends:\n", monthly_counts)


Daily News Frequency:
 date
2020-05-22    26
2020-05-26    28
2020-06-03    56
2020-06-05    62
dtype: int64

Day-of-week Trends (0=Monday, 6=Sunday):
 date
1.0    28
2.0    56
4.0    88
dtype: int64

Monthly Trends:
 date
2020-05     54
2020-06    118
Freq: M, dtype: int64


/tmp/ipykernel_8712/2949055833.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  monthly_counts = data.groupby(data['date'].dt.to_period('M')).size()


### **Text Analysis(Sentiment analysis & Topic Modeling)**

In [26]:
def analyze_sentiment(headline):
    analysis = TextBlob(headline)
    polarity = analysis.sentiment.polarity
    if polarity > 0.1:
        return 'positive'
    elif polarity < -0.1:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
data['sentiment'] = data['headline'].astype(str).apply(analyze_sentiment)

sentiment_counts = data['sentiment'].value_counts()
print("\nOverall Sentiment Distribution:\n", sentiment_counts)

In [ ]:
sentiment_over_time = data.groupby(data['date'].dt.date)['sentiment'].value_counts().unstack().fillna(0)
print("\nSentiment over Time:\n", sentiment_over_time)